In [1]:
import pandas as pd
from scipy.io import loadmat
import wfdb
import numpy as np
import os

from numpy import asarray
from numpy import save

In [2]:
import requests
import zipfile
from pathlib import Path


### download data

In [3]:
"""data_path = Path("data")
image_path = data_path / "12_lead_ECG_images"
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
"""

'data_path = Path("data")\nimage_path = data_path / "12_lead_ECG_images"\nif image_path.is_dir():\n    print(f"{image_path} directory exists.")\nelse:\n    print(f"Did not find {image_path} directory, creating one...")\n    image_path.mkdir(parents=True, exist_ok=True)\n'

In [4]:
# for downloading the data
"""import requests
from tqdm import tqdm

response = requests.get("https://physionet.org/static/published-projects/ecg-arrhythmia/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0.zip", stream=True)

# Total size in bytes.
total_size = int(response.headers.get('content-length', 0))
block_size = 1024 #1 Kibibyte

t=tqdm(total=total_size, unit='iB', unit_scale=True)

with open(data_path / "12_lead_ECG_images.zip", "wb") as f:
    for data in response.iter_content(block_size):
        t.update(len(data))
        f.write(data)
t.close()

if total_size != 0 and t.n != total_size:
    print("ERROR, something went wrong")"""

'import requests\nfrom tqdm import tqdm\n\nresponse = requests.get("https://physionet.org/static/published-projects/ecg-arrhythmia/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0.zip", stream=True)\n\n# Total size in bytes.\ntotal_size = int(response.headers.get(\'content-length\', 0))\nblock_size = 1024 #1 Kibibyte\n\nt=tqdm(total=total_size, unit=\'iB\', unit_scale=True)\n\nwith open(data_path / "12_lead_ECG_images.zip", "wb") as f:\n    for data in response.iter_content(block_size):\n        t.update(len(data))\n        f.write(data)\nt.close()\n\nif total_size != 0 and t.n != total_size:\n    print("ERROR, something went wrong")'

### annalysing data

In [5]:
path= "data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/WFDBRecords"

In [6]:
os.getcwd()

'/home/karansingh/Documents/summer-term/KARAN_ECG/data_prep'

In [7]:
path_1_list = os.listdir(path)
path_1_list

['22',
 '17',
 '10',
 '31',
 '11',
 '12',
 '05',
 '25',
 '19',
 '36',
 '40',
 '18',
 '20',
 '32',
 '01',
 '41',
 '35',
 '28',
 '27',
 '46',
 '21',
 '03',
 '02',
 '04',
 '34',
 '45',
 '09',
 '06',
 '39',
 '24',
 '07',
 '42',
 '15',
 '14',
 '30',
 '26',
 '33',
 '44',
 '43',
 '37',
 '13',
 '23',
 '16',
 '29',
 '38',
 '08']

In [8]:
# what we are going to do in disease_categorisation fxn
data = wfdb.rdsamp("data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/WFDBRecords/01/010/JS00018")# 
data

(array([[-0.059, -0.039,  0.02 , ...,  0.   ,  0.01 , -0.059],
        [-0.059, -0.039,  0.02 , ...,  0.   ,  0.01 , -0.059],
        [-0.059, -0.039,  0.02 , ...,  0.   ,  0.01 , -0.059],
        ...,
        [ 0.415,  0.244, -0.171, ...,  0.156,  0.327,  0.01 ],
        [ 0.415,  0.249, -0.166, ...,  0.176,  0.327,  0.005],
        [ 0.439,  0.259, -0.181, ...,  0.171,  0.332, -0.005]]),
 {'fs': 500,
  'sig_len': 5000,
  'n_sig': 12,
  'base_date': None,
  'base_time': None,
  'units': ['mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV',
   'mV'],
  'sig_name': ['I',
   'II',
   'III',
   'aVR',
   'aVL',
   'aVF',
   'V1',
   'V2',
   'V3',
   'V4',
   'V5',
   'V6'],
  'comments': ['Age: 61',
   'Sex: Male',
   'Dx: 426177001,59118001',
   'Rx: Unknown',
   'Hx: Unknown',
   'Sx: Unknown']})

In [9]:
data[1]['comments']

['Age: 61',
 'Sex: Male',
 'Dx: 426177001,59118001',
 'Rx: Unknown',
 'Hx: Unknown',
 'Sx: Unknown']

In [10]:
data[1]['comments'][2]

'Dx: 426177001,59118001'

In [11]:
data[1]['comments'][2].split()[1]

'426177001,59118001'

In [12]:
data[1]['comments'][2].split()[1].split(',')#disease_codes(data) output

['426177001', '59118001']

In [13]:
data[0].shape #### every ecg has 12 leads and 5000 points

(5000, 12)

## function defines

In [27]:
def disease_codes(data):
    disease_codes = data[1]['comments'][2].split()[1].split(',')
    return disease_codes

def disease_categorization(code):
                           #path_1_list:list,path : str):
    """we have some disease codes named code and we have to check in file eg: WFDBRecords/01/010/JS00018 file
    if data[1]['comments'][2].split()[1].split(',') contain the code then we have to append the data[0] to the list"""
    disease_list = []
    for i in range(len(path_1_list)):
        path_1 = path_1_list[i]#path_1 is the folder name inside the WFDBRecords
        relative_path_1 = path+'/'+path_1
        path_2_list = os.listdir(relative_path_1)#list of folder in the path_1


        for j in range(len(path_2_list)):# checking each folder in the path_1
            path_2 = path_2_list[j]


            path_to_dir = relative_path_1 + '/' + path_2 + '/'

            records = open(path_to_dir+'RECORDS','r').read().split()#WFDBRecords/path1/path2/RECORDS
            # reocords is a list of all the records in the path2 folder

            for k in range(len(records)):
                """ uncomment the commented paart to see which fie is currupted"""
                # print(path_1_list[i],path_2_list[j],records[k])
                # print(path_to_dir+'RECORDS')
                data = wfdb.rdsamp(path_to_dir+records[k])# explained above
                disease_codes_ = disease_codes(data)# return the desease codes of the record AS SHOWN ABOVE

                for l in range(len(disease_codes_)):
                    if disease_codes_[l] == str(code):# if the code is in the disease codes of the record

                        disease_list.append(data[0])
                

    
    return disease_list

def disease_codes(data):
    disease_codes = data[1]['comments'][2].split()[1].split(',')
    return disease_codes

def unique_disease_categorization(code):
                           #path_1_list:list,path : str):
    """we have some disease codes named code and we have to check in file eg: WFDBRecords/01/010/JS00018 file
    if data[1]['comments'][2].split()[1].split(',') contain the code then we have to append the data[0] to the list"""
    disease_list = []
    for i in range(len(path_1_list)):
        path_1 = path_1_list[i]#path_1 is the folder name inside the WFDBRecords
        relative_path_1 = path+'/'+path_1
        path_2_list = os.listdir(relative_path_1)#list of folder in the path_1


        for j in range(len(path_2_list)):# checking each folder in the path_1
            path_2 = path_2_list[j]


            path_to_dir = relative_path_1 + '/' + path_2 + '/'

            records = open(path_to_dir+'RECORDS','r').read().split()#WFDBRecords/path1/path2/RECORDS
            # reocords is a list of all the records in the path2 folder

            for k in range(len(records)):
                """ uncomment the commented paart to see which fie is currupted"""
                # print(path_1_list[i],path_2_list[j],records[k])
                # print(path_to_dir+'RECORDS')
                data = wfdb.rdsamp(path_to_dir+records[k])# explained above
                disease_codes_ = disease_codes(data)# return the desease codes of the record AS SHOWN ABOVE

                if len(disease_codes_) == 1: # for records with only one disease code
                    if disease_codes_[0] == str(code):# if the code is in the disease_codes_ then append the data[0] to the list
                        disease_list.append(data[0])
    return disease_list




In [15]:
path_to_disease_codes = 'data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/ConditionNames_SNOMED-CT.csv'
# it is a csv file containing the disease codes and their names
disease_codes_list = pd.DataFrame(pd.read_csv(path_to_disease_codes))

In [16]:
disease_codes_list
#snomed_ct is the disease code
# acronym is the name of the disease

,Acronym Name,Full Name,Snomed_CT
0,1AVB,1 degree atrioventricular block,270492004
1,2AVB,2 degree atrioventricular block,195042002
2,2AVB1,2 degree atrioventricular block(Type one),54016002
3,2AVB2,2 degree atrioventricular block(Type two),28189009
4,3AVB,3 degree atrioventricular block,27885002
...,...,...,...
58,SVT,Supraventricular Tachycardia,426761007
59,AT,Atrial Tachycardia,713422000
60,AVNRT,Atrioventricular Node Reentrant Tachycardia,233896004
61,AVRT,Atrioventricular Reentrant Tachycardia,233897008


In [17]:
from tqdm import tqdm

disease_name = disease_codes_list.get('Acronym Name')
disease_code = disease_codes_list.get('Snomed_CT')
disease_list = []  # IDK what this is for
code_list_len = []

for code in tqdm(disease_code, desc="Processing disease codes"):
    print(code)
    cc = unique_disease_categorization(code)
    print(len(cc))
    # disease_list.append(cc)  # list of records for each disease code
    code_list_len.append(len(cc))  # how many records are there for each disease code

Processing disease codes:   0%|          | 0/63 [00:00<?, ?it/s]

270492004


Processing disease codes:   0%|          | 0/63 [00:18<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
code_list_len

[1140,
 66,
 31,
 0,
 76,
 3,
 1545,
 1312,
 1063,
 853,
 244,
 162,
 76,
 366,
 3,
 771,
 771,
 75,
 11,
 240,
 240,
 240,
 647,
 1043,
 0,
 0,
 123,
 123,
 123,
 123,
 123,
 52,
 142,
 394,
 36,
 649,
 110,
 1668,
 801,
 1158,
 176,
 7043,
 2877,
 136,
 3,
 56,
 116,
 294,
 12,
 8,
 2,
 72,
 16559,
 8125,
 1780,
 7254,
 8060,
 2550,
 724,
 297,
 0,
 26,
 2]

In [ ]:
for items in range(len(code_list_len)):
    if code_list_len[items] >= 5000:
        print(code_list_len[items])
        print(disease_name[items])
        print(disease_code[items],"\n")

7043
TWC
164934002 

16559
SB
426177001 

8125
SR
426783006 

7254
ST
427084000 

8060
AF
164890007 



# data save

In [24]:
SB_code = 426177001
SR_code = 426783006
ST_code = 427084000
AF_code = 164890007
TWC_code = 164934002

In [ ]:
# disease_AF= disease_categorization(AF_code)
# print(f"len of disease_AF: {len(disease_AF)}")
# disease_TWC = disease_categorization(TWC_code)
# print(f"len of disease_TWC: {len(disease_TWC)}")

# disease_AF= unique_disease_categorization(AF_code)
# print(f"len of only disease_AF: {len(disease_AF)}")
# disease_TWC = unique_disease_categorization(TWC_code)
# print(f"len of only disease_TWC: {len(disease_TWC)}")



len of disease_AF: 8060
len of disease_TWC: 7043
len of only disease_AF: 1483
len of only disease_TWC: 0


In [33]:
disease_SR = unique_disease_categorization(SR_code)
print(f"len of only disease_SR: {len(disease_SR)}")
disease_SB = unique_disease_categorization(SB_code)
print(f"len of only disease_SB: {len(disease_SB)}")
disease_ST = unique_disease_categorization(ST_code)
print(f"len of only disease_ST: {len(disease_ST)}")

len of only disease_SR: 5908
len of only disease_SB: 8909
len of only disease_ST: 3223


In [34]:
SB_code = 426177001
SR_code = 426783006
ST_code = 427084000
import random

print(len(disease_SB))
disease_SB_cut = disease_SB[0:5000]
disease_SB_cut_random=random.sample(disease_SB,5000)
data = asarray(disease_SB_cut)
data_random = asarray(disease_SB_cut_random)
print(1)
save('unq_disease_SB.npy', data)
print(3)
save('unq_disease_SB_random.npy', data_random)
print(2)
del disease_SB_cut,disease_SB_cut_random,data,data_random,disease_SB


print(len(disease_SR))
disease_SR_cut = disease_SR[0:5000]
disease_SR_cut_random=random.sample(disease_SR,5000)
data = asarray(disease_SR_cut)
data_random = asarray(disease_SR_cut_random)
save('unq_disease_SR.npy', data)
save('unq_disease_SR_random.npy', data_random)
del disease_SR_cut,disease_SR_cut_random,data,data_random,disease_SR


disease_ST_cut = disease_ST[0:3000]
disease_ST_cut_random=random.sample(disease_ST,3000)
data = asarray(disease_ST_cut)
data_random = asarray(disease_ST_cut_random)
save('unq_disease_ST.npy', data)
save('unq_disease_ST_random.npy', data_random)
del disease_ST_cut,disease_ST_cut_random,data,data_random,disease_ST






print("done done")

8909
1
3
2
5908
done done


In [35]:
""" u can use code_list_len directly by uncommenting """
#code_list_len=[1140,
#  66,
#  31,
#  0,
#  76,
#  3,
#  1545,
#  1312,
#  1063,
#  853,
#  244,
#  162,
#  76,
#  366,
#  3,
#  771,
#  771,
#  75,
#  11,
#  240,
#  240,
#  240,
#  647,
#  1043,
#  0,
#  0,
#  123,
#  123,
#  123,
#  123,
#  123,
#  52,
#  142,
#  394,
#  36,
#  649,
#  110,
#  1668,
#  801,
#  1158,
#  176,
#  7043,
#  2877,
#  136,
#  3,
#  56,
#  116,
#  294,
#  12,
#  8,
#  2,
#  72,
#  16559,
#  8125,
#  1780,
#  7254,
#  8060,
#  2550,
#  724,
#  297,
#  0,
#  26,
#  2]

' u can use code_list_len directly by uncommenting '